In [3]:
import swmmAPI_v2 as sm
from pyswmm import Simulation, Links, Nodes


inpF = 'NoWestside_withISDs_RainEvents_NoControl.inp'
outF = inpF[:-3]+'out'


model = sm.swmmINP(inpF,offset=0.0)
model.set_dicts()
model.set_geo_dicts()

In [ ]:
model.conduits
model.junctions
model.options

In [4]:
print(dir(model))

['Map', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_min_slope', '_sections', 'args', 'calc_datum_conversion', 'calc_qfull', 'calc_slope', 'calc_storage_vol', 'conduits', 'controls', 'convert', 'coords', 'curves', 'evaporation', 'headers', 'infiltration', 'inflows', 'inpF', 'junctions', 'kwargs', 'losses', 'make_conduit_dictionary', 'make_controls_dict', 'make_coords_dict', 'make_curves_dictionary', 'make_evap_dict', 'make_infiltration_dict', 'make_inflows_dict', 'make_junction_dictionary', 'make_losses_dict', 'make_map_dict', 'make_options_dictionary', 'make_orifice_dictionary', 'make_outfall_dictionary', 'make_polygons_dict', 'make_pump_dictionary', 'make_raingauges_dict', 'make_sections'

In [2]:
def raingauges_from_subcatchment(subcatchmentDict,raingauges,rg_format,interval,SCF,source1):
    for sub in subcatchmentDict:
        subcatchmentDict[sub]['rain_gage'] = sub
        
        raingauges[sub] = {
            'format' : rg_format,
            'interval' : interval,
            'SCF' : SCF,
            'source1' : source1,
            'source2' : sub
        }

In [3]:
def timeseries_from_raingauges(raingauges,timeseries):
    for sub in subcatchmentDict:
        try: 
            if timeseries[sub]:
                pass
#                 print(sub + ' exists')

        except:
            timeseries[sub] = {
                'Date': [],
                'Time': [],
                'Value': [],
            }

def add_timeseries_info(timeseries,name,date,time,value):
    timeseries[name]['Date'].append(date)
    timeseries[name]['Time'].append(time)
    timeseries[name]['Value'].append(value)

In [4]:
raingauges_from_subcatchment(subcatchmentDict,raingauges,'INTENSITY',0.25,1.0,'TIMESERIES')
timeseries_from_raingauges(raingauges,timeseries)

In [7]:
for name in timeseries:
    # Check on if the raingauge/timeseries is a direct link to the subcatchment
    for i in range(0,5):
        if name in subcatchmentDict.keys(): 
            # input specific values into the timeseries... incorporate other processes here.
            date = ''
            time = i * 0.25
            value = i * 0.1
            add_timeseries_info(timeseries,name,date,time,value)

In [9]:
def write_timeseries(timeseries,sublist):
    with open('timeseries.txt','w') as f:
        f.write('[TIMESERIES]\n')
        f.write(';;Name           Date       Time       Value     \n')
        f.write(';;-------------- ---------- ---------- ----------\n')
        
        for ts in timeseries:
            if ts in sublist:
                wrt = timeseries[ts]
                for d,t,v in zip(wrt['Date'],wrt['Time'],wrt['Value']):
                    f.write(ts + '\t \t' + str(d) + '\t \t' + str(t) + '\t \t' + str(v) + '\n')
                    
                f.write(';\n')
    f.close()

In [23]:
def write_raingauges(raingauges,sublist):
    with open('raingauges.txt','w') as f:
        f.write('[RAINGAGES]\n')
        f.write(';;Name           Format    Interval SCF      Source \n')
        f.write(';;-------------- --------- ------ ------ ----------\n')
        
        for gage in raingauges:
            if gage in sublist:
                rg = raingauges[gage]
                f.write(gage + '\t\t\t' + str(rg['format']) + '\t' + str(rg['interval']) + '\t' + str(rg['SCF']) + '\t' + str(rg['source1']) + '\t' + str(rg['source2']) + '\n')

In [39]:
def write_subcatchments(subcatchmentDict):
    with open('subcatchments.txt','w') as f:
        f.write('[SUBCATCHMENTS]\n')
        f.write(';;Name           Rain Gage        Outlet           Area     %Imperv  Width    %Slope   CurbLen  SnowPack\n')
        f.write(';;-------------- ---------------- ---------------- -------- -------- -------- -------- -------- ----------------\n')
        
        for sub in subcatchmentDict:
            s = subcatchmentDict[sub]
            f.write(sub + '\t\t' + s['rain_gage'] + '\t\t' + s['outlet'] + '\t\t' + str(s['area']) + '\t' + str(s['per_imperv']) + '\t' + str(s['width']) + '\t' + str(s['slope']) + '\t' + str(s['curblen']) + '\n')
        

In [5]:
from pyproj import Proj, transform
from shapely.geometry import Polygon
from descartes import PolygonPatch

inputEPSG = Proj(init='epsg:6498')
outputEPSG = Proj(init='epsg:4326') # WGS84

for key in polygons:
    polygons[key]['lat'] = []
    polygons[key]['lon'] = []
    polygons[key]['shapely'] = []
    for x,y in zip(polygons[key]['x'],polygons[key]['y']):
        lat,lon = transform(inputEPSG,outputEPSG,x,y)
        
        polygons[key]['lat'].append(lat)
        polygons[key]['lon'].append(lon)
        polygons[key]['shapely'].append((lat,lon))
    
    polygons[key]['polygon'] = Polygon(polygons[key]['shapely']) 

In [3]:
import pickle

pickleOut = 'NoWestide_geo.p'

saveDict = {
    'polygons': polygons,
    'coords' : coords,
    'maps' : maps,
    'verts' : verts,
}

with open(pickleOut,'wb') as f:
    pickle.dump(saveDict,f)

In [25]:
shape = Polygon(polygons[key]['shapely'])
shape.bounds

(-83.21442564842441, 42.504508896212506, -83.11112529164289, 42.56078390964576)